<div style='float: right'><img src='pic/skeleton.png'/></div>
## <div id='nskeleton' />スケルトン

In [ ]:
import numpy as np
from itertools import product, takewhile
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
*.*...*
*.*.*.*
....*.*
*.***..
*...***
*.*....""".split()
wds = """\
ウメ
キク
サクラ
サツキ
ツバキ
ガーベラ
キキヨウ
クチナシ
チューリツプ""".split()
nw, nh = len(data[0]), len(data)

### 問題
* 1つの白マスには0から9までの数字が1つだけ入ります
* 連続した白マスの数字をつなげると、与えられたナンバーの1つになります
* 各ナンバーはそのようにして一度しか出ません。
  * 異なる場所にある連続した白マスから同じナンバーは出ません
* 与えられたナンバーは全て盤面の中に見つかります
* <a href='な.ナンバースケルトン.ipynb'>参考</a>

### 変数
* 省略

### 制約
* 省略

In [ ]:
# 文字->index of dic2, index->文字, 文字数->(単語リスト,位置リスト)
dic1, dic2, dic3 = {}, {}, {}
for wd in wds:
    for c in wd:
        if c not in dic1:
            dic1[c], dic2[len(dic2)] = len(dic2), c
    dic3.setdefault(len(wd), ([], []))[0].append(wd)
M = len(dic1)
chk = lambda yx: 0<=yx[0]<nh and 0<=yx[1]<nw and data[yx[0]][yx[1]]!='*'
def adddc(sp):
    if len(sp) > 1:
        dic3[len(sp)][1].append(sp)
m = LpProblem()
v = addvars(nh, nw)
for i, j in product(range(nh), range(nw)):
    if i == 0 or data[i-1][j] == '*':
        adddc(list(takewhile(chk, [(i+k,j) for k in range(nw)])))
    if j == 0 or data[i][j-1] == '*':
        adddc(list(takewhile(chk, [(i,j+k) for k in range(nh)])))
for nl, (wl, pl) in dic3.items():
    nc = len(wl)
    w = np.array(addbinvars(nc, nc))
    for i in range(nc):
        m += lpSum(w[i,:]) == 1
        m += lpSum(w[:,i]) == 1
        for j in range(nc):
            wd = wl[j]
            for k in range(nl):
                (y, x), a = pl[i][k], dic1[wd[k]]
                m += v[y][x] <= a + M * (1 - w[i,j])
                m += v[y][x] >= a - M * (1 - w[i,j])
%time m.solve()
r = np.vectorize(value)(v)
print('\n'.join(' '.join(s) for s in np.vectorize(
    lambda x: dic2[int(x)] if x is not None else '  ')(r)))